In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re

In [2]:
# https://www.boxofficemojo.com/intl/?ref_=bo_nb_hm_tab

BASE_URL = "https://www.boxofficemojo.com"
INIT_LINK = "/intl/"

INIT_PARAMS = {
    'ref_': "bo_nb_hm_tab"
}

INIT_HEADERS = {
    'User-Agent': UserAgent().random,
    'Accept': "*/*"
}

In [3]:
session = requests.session() # для оптимизации повторных запросов

response = session.get(BASE_URL + INIT_LINK, params=INIT_PARAMS, headers=INIT_HEADERS)

soup = BeautifulSoup(response.text, features="html.parser")

soup.find("a")

<a class="a-link-normal" href="/?ref_=bo_nb_in_mojologo"></a>

In [ ]:
rows = soup.find_all("tr")
rePat = re.compile("mojo-field-type")

cols = list(
    {
        'name': col.findChild().getText(),
        'class': next(filter(rePat.match, col.get("class")))
    }
    for col in rows[0].findChildren("th")
)

for col in filter(lambda el: el['name'] == "Area" or el['name'] == "Weekend" or el['name'] == "#1 Release", cols):
    col['func'] = (
        lambda field: (fc.getText(), BASE_URL + fc.get("href")) if (fc := field.findChild()) else None
    )

for col in filter(lambda el: el['name'] == "Releases", cols):
    col['func'] = (
        lambda field: int(field.getText()) if field else None
    )

for col in filter(lambda el: el['name'] == "Distributor", cols):
    col['func'] = (
        lambda field: (fc.getText().replace("\n", ""), fc.get("href")) if (fc := field.findChild()) else None
    )

for col in filter(lambda el: el['name'] == "Weekend Gross", cols):
    col['func'] = (
        lambda field: float(field.getText()[::-1]
                                            .replace(",", ".", 1)[::-1]
                                            .replace(",", "").replace("-", "0")
                                            .lstrip("$")) if field else None
    )

cols

In [65]:
rows = soup.find_all("tr")
rePat = re.compile("mojo-field-type")

cols = {
    next(filter(rePat.match, col.get("class"))): {
        'name': col.findChild().getText()
    }
    for col in rows[0].findChildren("th")
}

for cl, col in cols.items():
    col['func'] = (
        lambda field: (fc.getText(), BASE_URL + fc.get("href")) if (fc := field.findChild()) else None
    ) \
    if re.fullmatch(".*(area_id|date_interval|type-release)$", cl) else (
        lambda field: int(field.getText()) if field else None
    ) \
    if re.fullmatch(".*positive_integer$", cl) else (
        lambda field: (fc.getText().replace("\n", ""), fc.get("href")) if (fc := field.findChild()) else None
    ) \
    if re.fullmatch(".*studio$", cl) else (
        lambda field: float(field.getText()[::-1]
                                            .replace(",", ".", 1)[::-1]
                                            .replace(",", "").replace("-", "0")
                                            .lstrip("$")) if field else None
    ) \
    if re.fullmatch(".*money$", cl) else None

cols

{'mojo-field-type-area_id': {'name': 'Area',
  'func': <function __main__.<lambda>(field)>},
 'mojo-field-type-date_interval': {'name': 'Weekend',
  'func': <function __main__.<lambda>(field)>},
 'mojo-field-type-positive_integer': {'name': 'Releases',
  'func': <function __main__.<lambda>(field)>},
 'mojo-field-type-release': {'name': '#1 Release',
  'func': <function __main__.<lambda>(field)>},
 'mojo-field-type-studio': {'name': 'Distributor',
  'func': <function __main__.<lambda>(field)>},
 'mojo-field-type-money': {'name': 'Weekend Gross',
  'func': <function __main__.<lambda>(field)>}}

In [67]:
resData = []
rePat = re.compile("mojo-field-type")

for row in rows[1:]:
    fields = row.findChildren("td")
    if len(fields) == len(cols):
        resDict = {}
        for field in fields:
            cl = next(filter(rePat.match, field.get("class")))
            resDict[cols[cl]['name']] = cols[cl]['func'](field)
        resData.append(resDict)
resData

[{'Area': ('China',
   'https://www.boxofficemojo.com/weekend/by-year/?area=CN&ref_=bo_in_table_1'),
  'Weekend': ('Apr 5-7',
   'https://www.boxofficemojo.com/weekend/2024W14/?area=CN&ref_=bo_in_table_1'),
  'Releases': 2,
  '#1 Release': ('Godzilla x Kong: The New Empire',
   'https://www.boxofficemojo.com/release/rl2321514497/?ref_=bo_in_table_1'),
  'Distributor': ('Warner Bros.',
   'https://pro.imdb.com/company/co0002663/boxoffice/?area=CN&view=releases&ref_=mojo_in_table_1&rf=mojo_in_table_1'),
  'Weekend Gross': 21400.0},
 {'Area': ('Mexico',
   'https://www.boxofficemojo.com/weekend/by-year/?area=MX&ref_=bo_in_table_2'),
  'Weekend': ('Apr 5-7',
   'https://www.boxofficemojo.com/weekend/2024W14/?area=MX&ref_=bo_in_table_2'),
  'Releases': 1,
  '#1 Release': ('The First Omen',
   'https://www.boxofficemojo.com/release/rl1063354369/?ref_=bo_in_table_2'),
  'Distributor': ('Walt Disney Pictures',
   'https://pro.imdb.com/company/co0008970/boxoffice/?area=MX&view=releases&ref_=moj